In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [43]:
#Load Data
pd.set_option("display.max_rows", None)
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,25/11/2020 18:28:01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df = df.drop(columns='Timestamp')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 17 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Age                                                     1226 non-null   float64
 1   Gender                                                  1243 non-null   object 
 2   City                                                    1253 non-null   object 
 3   Position                                                1247 non-null   object 
 4   Total years of experience                               1237 non-null   object 
 5   Years of experience in Germany                          1221 non-null   object 
 6   Seniority level                                         1241 non-null   object 
 7   Your main technology / programming language             1126 non-null   object 
 8   Other technologies/programming languag

In [50]:
df['Your main technology / programming language'].value_counts()

Java                                                            195
Python                                                          192
JavaScript                                                       84
PHP                                                              69
C++                                                              40
C#                                                               31
Swift                                                            25
Kotlin                                                           24
Scala                                                            24
TypeScript                                                       23
Go                                                               23
Python                                                           21
.NET                                                             20
Ruby                                                             19
Other                                           

In [48]:
df['Your main technology / programming language'].replace('Php', "PHP", inplace=True)
df['Your main technology / programming language'].replace('php', "PHP", inplace=True)
df['Your main technology / programming language'].replace('java', "Java", inplace=True)
df['Your main technology / programming language'].replace('python', "Python", inplace=True)
df['Your main technology / programming language'].replace('Javascript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('javascript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Typescript', "TypeScript", inplace=True)
df['Your main technology / programming language'].replace('Android/Kotlin', "Kotlin", inplace=True)
df['Your main technology / programming language'].replace('JavaScript / typescript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Javascript / Typescript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Swift, Objective-C, iOS', "Swift", inplace=True)
df['Your main technology / programming language'].replace('Web developer', "Other", inplace=True)
df['Your main technology / programming language'].replace('DevOps', "Other", inplace=True)
df['Your main technology / programming language'].replace('none', "Other", inplace=True)
df['Your main technology / programming language'].replace('Pascal, VB.NET, C#', "Other", inplace=True)
df['Your main technology / programming language'].replace('Jira', "Other", inplace=True)
df['Your main technology / programming language'].replace('FBD', "Other", inplace=True)
df['Your main technology / programming language'].replace('NodeJS/TS', "Other", inplace=True)
df['Your main technology / programming language'].replace('Network Automation', "Other", inplace=True)
df['Your main technology / programming language'].replace('Oracle', "Other", inplace=True)
df['Your main technology / programming language'].replace('TypeScript, Kotlin', "TypeScript", inplace=True)
df['Your main technology / programming language'].replace('Clojure', "Other", inplace=True)
df['Your main technology / programming language'].replace('SAP BW / ABAP', "Other", inplace=True)
df['Your main technology / programming language'].replace('С#', "C#", inplace=True)
df['Your main technology / programming language'].replace('JavaScript/ES6', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('C++, Java, Embedded C', "C++", inplace=True)
df['Your main technology / programming language'].replace('Js', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Agile', "Other", inplace=True)
df['Your main technology / programming language'].replace('Python, statistics, SQL', "Python", inplace=True)
df['Your main technology / programming language'].replace('NLP, Python', "Python", inplace=True)
df['Your main technology / programming language'].replace('Java/Scala', "Java", inplace=True)
df['Your main technology / programming language'].replace('Computer Networking,  Network Security', "Other", inplace=True)
df['Your main technology / programming language'].replace('Java / Scala', "Java", inplace=True)
df['Your main technology / programming language'].replace('Javascript, Angular', "JavaScript", inplace=True)

df['Your main technology / programming language'].replace('-', "Other", inplace=True)
df['Your main technology / programming language'].replace('Kotlin/PHP', "Kotlin", inplace=True)
df['Your main technology / programming language'].replace('GCP', "Other", inplace=True)
df['Your main technology / programming language'].replace('Js, TypeScript, Angular', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Embedded C++', "C++", inplace=True)
df['Your main technology / programming language'].replace('.net', ".NET", inplace=True)

In [44]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,25/11/2020 18:28:01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score

numeric_features = ["Age", 'Yearly brutto salary (without bonus and stocks) in EUR']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

categorical_features = ["]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer( remainder='drop',
        transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

#Split data as normal
y = df[{"Yearly brutto salary (without bonus and stocks) in EUR", 'Yearly bonus + stocks in EUR'}]
X = df.drop(columns={"Yearly brutto salary (without bonus and stocks) in EUR", 'Yearly bonus + stocks in EUR'})

#Build pipeline
l1_model = Lasso()
l2_model = Ridge()
pipeline_steps = [('pre', preprocessor),('model', l1_model) ]
pipe = Pipeline(pipeline_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set

pipe.fit(X_train, y_train)
cvs = cross_val_score(pipe, X_train, y_train.ravel(), cv=5)
print('Training CrossVal Score:', cvs)
print("Average Train Score:", np.mean(cvs))
print('Testing score:', pipe.score(X_test, y_test))

SyntaxError: EOL while scanning string literal (<ipython-input-33-dd892350e22b>, line 14)

# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes